# ETA_Adjustment

Before you run you must complete the following steps. 

    Change shipping number header to "Logical Tracking ID"

    Dont forget to also update the date (Three of them) before the loops

In [1]:
import pandas as pd
import pyodbc
import numpy as np
from datetime import datetime, timedelta, date,time
import urllib

## Primary Query

In [2]:
conn=pyodbc.connect(r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
                    r'DBQ=//cpint_data/data/MERCHANDISING/ReBuyers Shared/Daily Updates - Purchasing.mdb;')
po_out_qry="Select * From [PO Out]"
po_pd=pd.read_sql_query(po_out_qry,conn)

In [3]:
wkend_pd = pd.read_excel('weekend_Receipts.xlsx',sheetname='Sheet1',index_col=2)
wkend_pd['TRANSACTION_DATE'] = wkend_pd['TRANSACTION_DATE'].apply(lambda x: x.strftime('%Y-%m-%d'))
wkend_pd['DT_IMPORTED'] = wkend_pd['DT_IMPORTED'].apply(lambda x: x.strftime('%Y-%m-%d'))
wkend_pd['Track_ID'] = wkend_pd['PO_NBR'].map(str)
wkend_pd['Track_ID'] = wkend_pd['Track_ID'].astype(str).str[:-2]
wkend_pd['Track_ID'] = wkend_pd['Track_ID'] + '-' + wkend_pd.index
fri_receipts_pd = wkend_pd.loc[wkend_pd['TRANSACTION_DATE'] == '2018-05-11']
wkend_pd.head()

,ORG,TRANSACTION_DATE,VERIFIED,PO_NBR,SKU,RECEIVE_QTY,RECEIVE_CARTON,DT_IMPORTED,Track_ID
Logical Tracking ID,,,,,,,,,
17-035136,DIL,2018-01-02,Y,6457613.0,61157,2352,2352,2018-01-03,6457613-17-035136
17-035136,DIL,2018-01-02,Y,6458564.0,61326,144,1,2018-01-03,6458564-17-035136
17-035136,DIL,2018-01-02,Y,6458918.0,95969,600,150,2018-01-03,6458918-17-035136
17-035136,DIL,2018-01-02,Y,6462553.0,61156,1000,5,2018-01-03,6462553-17-035136
17-035355,DIL,2018-01-02,Y,6455130.0,61870,850,850,2018-01-03,6455130-17-035355


## Past Receipts

In [42]:
conn_net = pyodbc.connect(r'DRIVER={NetezzaSQL};'
                          r'SERVER=PUREDATA_PRD;'
                          r'PORT=5480;'
                          r'DATABASE=DNA_PUB_INVENTORY;'
                          r'UID=DNA_fsinohui;'
                          r'PWD=fsinohui')

In [43]:
receipts_qry = ('SELECT WMS_RECEIPTS_ALLORG.SHIPMENT_NUMBER, WMS_RECEIPTS_ALLORG.ORG, '
                'WMS_RECEIPTS_ALLORG.TRANSACTION_DATE, WMS_RECEIPTS_ALLORG.VERIFIED, '
                'WMS_RECEIPTS_ALLORG.PO_NBR, WMS_RECEIPTS_ALLORG.SKU, '
                'WMS_RECEIPTS_ALLORG.RECEIVE_QTY, WMS_RECEIPTS_ALLORG.RECEIVE_CARTON,' 
                'WMS_RECEIPTS_ALLORG.DT_IMPORTED '
                'FROM WMS_RECEIPTS_ALLORG '
                "WHERE ((WMS_RECEIPTS_ALLORG.TRANSACTION_DATE)>'3/1/2018' );" 
                )

In [44]:
# Run Netezza query
wkend_pd=pd.read_sql_query(receipts_qry,conn_net, index_col='SHIPMENT_NUMBER')
wkend_pd['TRANSACTION_DATE'] = wkend_pd['TRANSACTION_DATE'].apply(lambda x: x.strftime('%Y-%m-%d'))
wkend_pd['DT_IMPORTED'] = wkend_pd['DT_IMPORTED'].apply(lambda x: x.strftime('%Y-%m-%d'))
wkend_pd['Track_ID'] = wkend_pd['PO_NBR'].map(str)
wkend_pd['Track_ID'] = wkend_pd['Track_ID'].astype(str).str[:-2]
wkend_pd['Track_ID'] = wkend_pd['Track_ID'] + '-' + wkend_pd.index
fri_receipts_pd = wkend_pd.loc[wkend_pd['TRANSACTION_DATE'] == '2018-05-11']
wkend_pd.head()

,ORG,TRANSACTION_DATE,VERIFIED,PO_NBR,SKU,RECEIVE_QTY,RECEIVE_CARTON,DT_IMPORTED,Track_ID
SHIPMENT_NUMBER,,,,,,,,,
17-041170,DIL,2018-03-02,Y,6439523,68792,1800,300,2018-03-03,64395-17-041170
17-041170,DIL,2018-03-02,Y,6448720,92553,2200,220,2018-03-03,64487-17-041170
17-041170,DIL,2018-03-02,Y,6452272,68805,3000,500,2018-03-03,64522-17-041170
17-042157,DIL,2018-03-02,Y,6478112,60496,2400,1200,2018-03-03,64781-17-042157
17-042157,DIL,2018-03-02,Y,6478112,63095,2400,1200,2018-03-03,64781-17-042157


# Just for Mondays Data

## Only use if not Monday

In [4]:
#Just for Mondays Data
conn=pyodbc.connect(r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
                    r'DBQ=//hftdata02/sysdata02/RedCat/Python/ETA_Adjustment/Daily Updates - Purchasing.accdb;')
po_out_qry="Select * From [PO Out]"
po_pd=pd.read_sql_query(po_out_qry,conn)
len(po_pd)

36048

In [4]:
# so you dont have to keep pulling the data
revert = po_pd

In [5]:
po_pd = revert

## Container Queries

In [6]:
cont_hdr_qry = ("Select [Logical Tracking ID],Vessel,Status,ETD_Date, ETA_Date,POL,POD,[Container Size], "
                "Cartons,Location,Container,SCAC,IN_DATE, OUT_DATE,Date_Available, Date_Ordered, [Date_Out Gated], "
                "[Date_1st Arrived], [On Priority], Date_Empty, [Picked up for_Term], Date_Terminated "
                "From [cont hdr]")

cont_hdr_pd=pd.read_sql_query(cont_hdr_qry,conn, index_col='Logical Tracking ID')

cont_dtl_qry = "Select * From [Cont Dtl]"

cont_dtl_pd=pd.read_sql_query(cont_dtl_qry,conn,index_col='Logical Tracking ID')

In [7]:
cont_pd = cont_dtl_pd.join(cont_hdr_pd, lsuffix='_l', rsuffix='_r')
cont_pd['Track_ID'] = cont_pd['PO Number']+'-'+cont_pd.index
cont_pd.reset_index(inplace=True)
cont_pd['Ship Qty'] = cont_pd[cont_pd['Ship Qty'] > 0]['Ship Qty']
cont_pd = cont_pd[np.isfinite(cont_pd['Ship Qty'])]
cont_pd.reset_index()
cont_pd.set_index('Logical Tracking ID',inplace=True)
cont_pd['ID'] = cont_pd['Item'].map(str)+'-'+cont_pd['PO Number'].map(str)
cont_pd['Ship To Location']='DILLON'
cont_pd.loc[cont_pd['Org']=='MOR','Ship To Location']='MORENO VALLEY'
cont_pd.loc[cont_pd['Org']=='CAM','Ship To Location']='CAMARILLO'
#Filters out all of the lines that were received since the file was last updated. 
cont_pd = cont_pd[~(cont_pd['Track_ID'].isin(wkend_pd['Track_ID']))]

In [8]:
po_filter_pd=cont_pd[['Item','PO Number','Ship Qty']]
po_filter_pd['ID'] = po_filter_pd['Item'].map(str)+'-'+po_filter_pd['PO Number'].map(str)
po_filter_pd.reset_index(inplace=True)
cont_grouped = po_filter_pd.groupby('ID').sum()
cont_grouped = cont_grouped.rename(columns={'Item':'SKU','Ship Qty':'ship_qty'})

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
po_pd['ID'] = po_pd['SKU'].map(str)+'-'+po_pd['Legacy PO NUM'].map(str)
po_pd.set_index('ID',inplace=True)
po_pd = po_pd.join(cont_grouped, how='left', rsuffix='_r')
po_pd['ship_qty'].fillna(0, inplace=True)
po_pd['ship_qty'] = po_pd['ship_qty'].map(int)
po_pd['Adj_Qty_raw']=po_pd['PO Open Qty']-po_pd['ship_qty']
po_pd['Adj_Qty'] = np.where(po_pd["Adj_Qty_raw"] <0,0,po_pd['Adj_Qty_raw'])
#po_pd['Adj_Qty'] = po_pd[po_pd['Adj_Qty'] > 0]['Adj_Qty']
po_pd = po_pd[np.isfinite(po_pd['Adj_Qty'])]
po_pd = po_pd.sort_values('Adj_Qty_raw',ascending=True)
len(po_pd)

36155

In [10]:
cont_pd.reset_index(inplace=True)
cont_pd=cont_pd[['Logical Tracking ID','Item','PO Number','Ord Qty', 'Ship To Location','Ship Qty','ETD_Date','Supplier Number','ETA_Date','POD']]
cont_pd['FOB-Cost']=0
cont_pd['PO Open Qty'] = cont_pd['Ship Qty'].values
cont_pd = cont_pd.rename(columns={'Item':'SKU','PO Number':'PO Number', 'Ship To Location':'Ship To Location',
                                  'POD':'POD','Ship Qty':'Shipped Qty','ETD_Date':'Approved_Date','Supplier Number':'Vendor No',
                                  'ETA_Date':'WE_Date'})
cont_pd = cont_pd.sort_values('WE_Date',ascending=True)
cont_pd['day_of_week']=cont_pd['WE_Date'].dt.dayofweek
cont_pd.head()

,Logical Tracking ID,SKU,PO Number,Ord Qty,Ship To Location,Shipped Qty,Approved_Date,Vendor No,WE_Date,POD,FOB-Cost,PO Open Qty,day_of_week
0,18-003229,96650,6469389,5000,MORENO VALLEY,5000.0,2018-01-06,36291,2018-02-19,LA/LB,0,5000.0,0
1,18-003229,36905,6474680,828,MORENO VALLEY,828.0,2018-01-06,36291,2018-02-19,LA/LB,0,828.0,0
2,18-003229,96651,6472208,3000,MORENO VALLEY,3000.0,2018-01-06,36291,2018-02-19,LA/LB,0,3000.0,0
3,18-003229,36904,6472208,1500,MORENO VALLEY,1500.0,2018-01-06,36291,2018-02-19,LA/LB,0,1500.0,0
38,18-006878,63095,6483340,2400,DILLON,2400.0,2018-03-10,36359,2018-04-13,CHARLESTON,0,2400.0,4


In [11]:
fri={0:4,1:3,2:2,3:1,4:0,5:6,6:5}
start_date = datetime(2018,5,11) #previous Friday
fri_lst =[]
for i, row in cont_pd.iterrows():
    fri_date = (cont_pd.get_value(i,'WE_Date',takeable=False)
                + timedelta(days=fri[cont_pd.get_value(i,'day_of_week',takeable=False)]))
    if fri_date <= start_date:
        fri_date=start_date
    fri_lst.append(fri_date)
se = pd.Series(fri_lst)    
cont_pd['Fri_date'] = se.values
grouped_df = cont_pd.groupby( [ 'Logical Tracking ID','POD', 'Fri_date'] ).size().to_frame(name = 'count').reset_index()
grouped_df = grouped_df.groupby(['POD', 'Fri_date']).size().to_frame(name = 'count').reset_index()
grouped_df

,POD,Fri_date,count
0,CHARLESTON,2018-05-11,479
1,CHARLESTON,2018-05-18,406
2,CHARLESTON,2018-05-25,324
3,CHARLESTON,2018-06-01,342
4,CHARLESTON,2018-06-08,324
5,CHARLESTON,2018-06-15,272
6,CHARLESTON,2018-06-22,108
7,CHARLESTON,2018-06-29,7
8,CHARLESTON,2018-07-06,1
9,LA/LB,2018-05-11,544


In [12]:
final_cont_df = pd.DataFrame()
for i, row in grouped_df.iterrows():
    pod_value=grouped_df.get_value(i,'POD',takeable=False)
    fri_date_value=grouped_df.get_value(i,'Fri_date',takeable=False)
    qty_value=grouped_df.get_value(i,'count',takeable=False)
    if fri_date_value == start_date:
        low_limit=fri_date_value-timedelta(days=500)
    else:
        low_limit=fri_date_value-timedelta(days=6)
    sub_pd = cont_hdr_pd[(cont_hdr_pd['POD'] == pod_value) & 
                         (cont_hdr_pd['ETA_Date'] <= fri_date_value)& 
                         (cont_hdr_pd['ETA_Date'] >= low_limit)]
    sub_pd = sub_pd.sort_values('ETA_Date',ascending=True)
    qtysum=0
    finaldatelst=[]
    if fri_date_value <= start_date:
        limit = 1
    elif pod_value == 'WILMINGTON':
        limit = 0.5
    else:
        limit = 0.7
    
    for j, rowj in sub_pd.iterrows():
        qtysum=qtysum + 1
        percent = float(qtysum/qty_value)
        if percent<=limit:
            final_date=fri_date_value
        else:
            final_date=fri_date_value + timedelta(days=7)
        finaldatelst.append(final_date)
    se_sub = pd.Series(finaldatelst)    
    sub_pd['Final_date'] = se_sub.values
    final_cont_df = final_cont_df.append(sub_pd)
final_cont_df.reset_index(inplace=True)
grouped_final_df = final_cont_df.groupby( [ 'Logical Tracking ID','POD', 'Final_date'] ).size().to_frame(name = 'count').reset_index()
grouped_final_df = final_cont_df.groupby(['POD', 'Final_date']).size().to_frame(name = 'count').reset_index()

final_cont_df.to_csv(r'\\hftdata02\sysdata02\RedCat\python\ETA_Adjustment\cont_hdr_adjusted.csv', sep=',',index=False)

In [13]:
grouped_df = final_cont_df.groupby( [ 'POD', 'Final_date'] ).size().to_frame(name = 'count').reset_index()
grouped_df

,POD,Final_date,count
0,CHARLESTON,2018-05-11,479
1,CHARLESTON,2018-05-18,317
2,CHARLESTON,2018-05-25,348
3,CHARLESTON,2018-06-01,337
4,CHARLESTON,2018-06-08,329
5,CHARLESTON,2018-06-15,288
6,CHARLESTON,2018-06-22,157
7,CHARLESTON,2018-06-29,37
8,CHARLESTON,2018-07-06,3
9,CHARLESTON,2018-07-13,1


In [14]:
final_cont_df = final_cont_df.set_index('Logical Tracking ID')
cont_pd = cont_dtl_pd.join(final_cont_df, rsuffix='_r')
cont_pd['Track_ID'] = cont_pd['PO Number']+'-'+cont_pd.index
cont_pd.reset_index(inplace=True)
cont_pd['Ship Qty'] = cont_pd[cont_pd['Ship Qty'] > 0]['Ship Qty']
cont_pd = cont_pd[np.isfinite(cont_pd['Ship Qty'])]
po_pd = po_pd[['SKU', 'PO Number', 'Adj_Qty', 'Ship To Location',       
       'Shipped Qty', 'Approved_Date', 'Vendor No', 'FOB-Cost', 'WE_Date']]
po_pd['POD']='NOT SHIPPED'
po_pd=po_pd.rename(columns={'Adj_Qty':'PO Open Qty'})
po_pd.head()

,SKU,PO Number,PO Open Qty,Ship To Location,Shipped Qty,Approved_Date,Vendor No,FOB-Cost,WE_Date,POD
ID,,,,,,,,,,
61716-6489602,61716,6489602,0,DILLON,64150,2018-03-22,36027,2.40,2018-05-18,NOT SHIPPED
69120-6497727,69120,6497727,0,DILLON,13500,2018-03-05,35181,6.11,2018-06-29,NOT SHIPPED
63822-6460970,63822,6460970,0,DILLON,0,2018-04-27,37041,1.63,2018-05-11,NOT SHIPPED
63890-6496808,63890,6496808,0,DILLON,4740,2018-01-22,36947,128.00,2018-05-18,NOT SHIPPED
63815-6475638,63815,6475638,0,DILLON,6120,2018-02-02,37041,0.51,2018-05-18,NOT SHIPPED


In [15]:
len(cont_pd)

12297

In [16]:
cont_pd['Ship To Location']='DILLON'
cont_pd.loc[cont_pd['Org']=='MOR','Ship To Location']='MORENO VALLEY'
cont_pd.loc[cont_pd['Org']=='CAM','Ship To Location']='CAMARILLO'
#cont_pd.head()

In [17]:
#Filters out all of the lines that were received since the file was last updated. 
cont_pd = cont_pd[~(cont_pd['Track_ID'].isin(wkend_pd['Track_ID']))]
len(cont_pd)

12065

In [18]:
cont_pd=cont_pd[['Item','PO Number','Ord Qty', 'Ship To Location','Ship Qty','ETD_Date','Supplier Number','POD','Final_date']]
cont_pd = cont_pd.rename(columns={'Item':'SKU','PO Number':'PO Number','Ord Qty':'PO Open Qty', 'Ship To Location':'Ship To Location',
                                  'POD':'POD','Ship Qty':'Shipped Qty','ETD_Date':'Approved_Date','Supplier Number':'Vendor No',
                                  'Final_date':'ETA_Date'})

In [19]:
cont_pd['FOB-Cost']=0
cont_pd['WE_Date']=cont_pd['ETA_Date'] + timedelta(days=7)
cont_pd['PO Open Qty'] = cont_pd['Shipped Qty'].values
cont_pd['ID'] = cont_pd['SKU'].map(str)+'-'+cont_pd['PO Number'].map(str)
cont_pd.head()

,SKU,PO Number,PO Open Qty,Ship To Location,Shipped Qty,Approved_Date,Vendor No,POD,ETA_Date,FOB-Cost,WE_Date,ID
0,96650,6469389,5000.0,MORENO VALLEY,5000.0,2018-01-06,36291,LA/LB,2018-05-11,0,2018-05-18,96650-6469389
1,36905,6474680,828.0,MORENO VALLEY,828.0,2018-01-06,36291,LA/LB,2018-05-11,0,2018-05-18,36905-6474680
2,96651,6472208,3000.0,MORENO VALLEY,3000.0,2018-01-06,36291,LA/LB,2018-05-11,0,2018-05-18,96651-6472208
3,36904,6472208,1500.0,MORENO VALLEY,1500.0,2018-01-06,36291,LA/LB,2018-05-11,0,2018-05-18,36904-6472208
7,97545,6472874,972.0,DILLON,972.0,2018-03-05,36494,CHARLESTON,2018-05-11,0,2018-05-18,97545-6472874


In [20]:
cont_pd.to_csv(r'\\hftdata02\sysdata02\RedCat\python\ETA_Adjustment\cont_pd_adjusted.csv', sep=',',index=False)

In [21]:
grouped_df = cont_pd.groupby( [ 'POD', 'ETA_Date'] ).size().to_frame(name = 'count').reset_index()
#grouped_df

In [22]:
inbound_pd = po_pd.append(cont_pd)
#inbound_pd.head()

In [23]:
inbound_pd.head()

,Approved_Date,ETA_Date,FOB-Cost,ID,PO Number,PO Open Qty,POD,SKU,Ship To Location,Shipped Qty,Vendor No,WE_Date
61716-6489602,2018-03-22,NaT,2.40,NaN,6489602,0.0,NOT SHIPPED,61716,DILLON,64150.0,36027,2018-05-18
69120-6497727,2018-03-05,NaT,6.11,NaN,6497727,0.0,NOT SHIPPED,69120,DILLON,13500.0,35181,2018-06-29
63822-6460970,2018-04-27,NaT,1.63,NaN,6460970,0.0,NOT SHIPPED,63822,DILLON,0.0,37041,2018-05-11
63890-6496808,2018-01-22,NaT,128.00,NaN,6496808,0.0,NOT SHIPPED,63890,DILLON,4740.0,36947,2018-05-18
63815-6475638,2018-02-02,NaT,0.51,NaN,6475638,0.0,NOT SHIPPED,63815,DILLON,6120.0,37041,2018-05-18


In [24]:
po_pd.head()

,SKU,PO Number,PO Open Qty,Ship To Location,Shipped Qty,Approved_Date,Vendor No,FOB-Cost,WE_Date,POD
ID,,,,,,,,,,
61716-6489602,61716,6489602,0,DILLON,64150,2018-03-22,36027,2.40,2018-05-18,NOT SHIPPED
69120-6497727,69120,6497727,0,DILLON,13500,2018-03-05,35181,6.11,2018-06-29,NOT SHIPPED
63822-6460970,63822,6460970,0,DILLON,0,2018-04-27,37041,1.63,2018-05-11,NOT SHIPPED
63890-6496808,63890,6496808,0,DILLON,4740,2018-01-22,36947,128.00,2018-05-18,NOT SHIPPED
63815-6475638,63815,6475638,0,DILLON,6120,2018-02-02,37041,0.51,2018-05-18,NOT SHIPPED


### Correct for partial Receipts

In [65]:
'''Here is where you will compare the raw PO Out with the new inbound_pd groupbys. '''

'Here is where you will compare the raw PO Out with the new inbound_pd groupbys. '

In [145]:
po_check_pd = po_pd
po_check_pd.reset_index(inplace=True)
po_check_pd = po_check_pd[['ID','SKU', 'PO Number', 'PO Open Qty']]
po_check_pd = po_check_pd.groupby(['ID','SKU','PO Number']).sum()
po_check_pd['PO Open Qty'] = po_check_pd[po_check_pd['PO Open Qty'] > 0]['PO Open Qty']
po_check_pd = po_check_pd[np.isfinite(po_check_pd['PO Open Qty'])]
po_check_pd.reset_index(inplace=True)
po_check_pd.set_index('ID', inplace=True)
i = '61726-6497323'
#for i, row in po_check_pd.iterrows():
open_value=po_check_pd.get_value(i,'PO Open Qty',takeable=False)
sku_value=po_check_pd.get_value(i,'SKU',takeable=False)
num_value=po_check_pd.get_value(i,'PO Number',takeable=False)
po_check_pd.head()

KeyError: '61726-6497323'

In [115]:
open_po_out_value=po_check_pd.get_value(i,'PO Open Qty',takeable=False)
open_po_out_value

2352.0

In [117]:
inbound_check = inbound_pd[(inbound_pd['PO Number']==num_value) & (inbound_pd['SKU']==sku_value)]
#inbound_check = inbound_pd['SKU']==sku_value
inbound_check

,Approved_Date,ETA_Date,FOB-Cost,ID,PO Number,PO Open Qty,POD,SKU,Ship To Location,Shipped Qty,Vendor No,WE_Date
61726-6497323,2018-01-22,NaT,15.07,NaN,6497323,2352.0,NOT SHIPPED,61726,MORENO VALLEY,0.0,36146,2018-06-01


In [108]:
opens = []
open_qty = 0
total_open = 0
for ib, row in inbound_check.iterrows():
    open_py_value = inbound_check.get_value(i,'PO Open Qty',takeable=False)
    opens.append(open_qty)
    total_open = total_open + open_qty
if open_py_value/open_po_out_value == 1:
    print('all good in the hood')
else:
    print('no go joe')

all good in the hood


### Final output preperations

In [ ]:
po_lst = ['PO_01','PO_02','PO_03','PO_04','PO_05','PO_06','PO_07','PO_08','PO_09',
          'PO_10','PO_11','PO_12','PO_13','PO_14','PO_15','PO_16','PO_17','PO_18','PO_19',
          'PO_20','PO_21','PO_22','PO_23','PO_24','PO_25','PO_26','PO_27','PO_28','PO_29',
          'PO_30','PO_31','PO_32','PO_33','PO_34','PO_35','PO_36','PO_37','PO_38','PO_39',
          'PO_40','PO_41','PO_42','PO_43','PO_44','PO_45','PO_46','PO_47','PO_48','PO_49',
          'PO_50','PO_51','PO_52']

po_no_lst = ['PO_NO_01','PO_NO_02','PO_NO_03','PO_NO_04','PO_NO_05','PO_NO_06','PO_NO_07','PO_NO_08','PO_NO_09',
             'PO_NO_10','PO_NO_11','PO_NO_12','PO_NO_13','PO_NO_14','PO_NO_15','PO_NO_16','PO_NO_17','PO_NO_18','PO_NO_19',
             'PO_NO_20','PO_NO_21','PO_NO_22','PO_NO_23','PO_NO_24','PO_NO_25','PO_NO_26','PO_NO_27','PO_NO_28','PO_NO_29',
             'PO_NO_30','PO_NO_31','PO_NO_32','PO_NO_33','PO_NO_34','PO_NO_35','PO_NO_36','PO_NO_37','PO_NO_38','PO_NO_39',
             'PO_NO_40','PO_NO_41','PO_NO_42','PO_NO_43','PO_NO_44','PO_NO_45','PO_NO_46','PO_NO_47','PO_NO_48','PO_NO_49',
             'PO_NO_50','PO_NO_51','PO_NO_52']
x = 0
for i in po_lst:
    x = x+1
    if x ==1:
        inbound_pd[i] = np.where(inbound_pd['WE_Date']<=start_date + timedelta(days=7*x),inbound_pd['PO Open Qty'],0)
    else:
        inbound_pd[i] = np.where(inbound_pd['WE_Date']==start_date + timedelta(days=7*x),inbound_pd['PO Open Qty'],0)
x = 0
for i in po_no_lst:
    x = x+1
    if x ==1:
        inbound_pd[i] = np.where(inbound_pd['WE_Date']<=start_date + timedelta(days=7*x),inbound_pd['PO Number'],0)
    else:
        inbound_pd[i] = np.where(inbound_pd['WE_Date']==start_date + timedelta(days=7*x),inbound_pd['PO Number'],0)

In [26]:
writer = pd.ExcelWriter('Inbound_consolidated_data.xlsx', engine='xlsxwriter')
inbound_pd.to_excel(writer,'inbound_pd')

In [3]:
test=pd.DataFrame()

In [ ]:
import pandas as pd

test=pd.DataFrame()
test.reset_index()
test.set_index(['Sku','Date'], inplace=True)